In [1]:
from data_util import load_embeddings, DMConfig, tfConfig
try:
    import _pickle as cPickle
except ImportError:
    import cPickle
import gzip
import os
import numpy as np
from defs import LBLS

ERROR:Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7feae265d730>)
Traceback (most recent call last):
  File "/home/tuckerleavitt/env/lib/python3.5/site-packages/tornado/ioloop.py", line 759, in _run_callback
    ret = callback()
  File "/home/tuckerleavitt/env/lib/python3.5/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/tuckerleavitt/env/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 263, in enter_eventloop
    self.eventloop(self)
  File "/home/tuckerleavitt/env/lib/python3.5/site-packages/ipykernel/eventloops.py", line 238, in loop_tk
    kernel.timer = Timer(doi)
  File "/home/tuckerleavitt/env/lib/python3.5/site-packages/ipykernel/eventloops.py", line 226, in __init__
    self.app = Tk()
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1871, in __init__
    self.tk = _tkinter.create(screenName, baseName, className, interactive, wantobjects, useTk, sync, use)
_tkint

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
config = DMConfig()
if os.path.exists(config.tok2id_path):
    with gzip.open(os.path.abspath(config.tok2id_path)) as f:
        tok2id = cPickle.load(f)
if os.path.exists(config.id2tok_path):
    with gzip.open(os.path.abspath(config.id2tok_path)) as f:
        id2tok = cPickle.load(f)

In [4]:
embeddings = load_embeddings(config, tok2id)

INFO:Initialized embeddings.


In [5]:
embeddings.shape

(10008, 50)

## Training

In [6]:
from tf_birnn_classifier import TfBiRNNClassifier
from defs import PROJECT_DIR

In [7]:
model = TfBiRNNClassifier(embedding=embeddings, eta=0.001, use_attn = True,
                           ckpts_prefix='{}/checkpoints/linear_model/birnn-model'.format(PROJECT_DIR),
                           summaries_dir='{}/summaries-birnn/'.format(PROJECT_DIR)
                          )
model.fit(tfConfig, restore_weights=False, batches_to_eval=100, max_iter=10)

INFO:Initializing data manager...
INFO:took 0 s
INFO:Building model...


Instructions for updating:
Use the retry module or similar alternatives.


Instructions for updating:
Use the retry module or similar alternatives.


fw outputs:  (?, 300, 50)
bw outputs:  (?, 300, 50)
all outputs:  (?, 300, 100)
all states:  (?, 100)
W_attn: (100, 25)
Instructions for updating:
dim is deprecated, use axis instead


Instructions for updating:
dim is deprecated, use axis instead
INFO:took 7 s
INFO:-- Initializingt new model
INFO:Starting epoch 1
INFO: == Evaluating on development data after batch 100
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.655, Recall: 0.655, F1: 0.655
INFO:-- Saving model
INFO: == Evaluating on development data after batch 200
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.682, Recall: 0.682, F1: 0.682
INFO:-- Saving model
INFO: == Evaluating on development data after batch 300
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.666, Recall: 0.666, F1: 0.666
INFO:-- Saving model
INFO: == Evaluating on development data after batch 400
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.651, Recall: 0.651, F1: 0.651
INFO:-- Saving model
INFO: == Evaluating on development data after batch 500
INFO:Ran 40 batches
INFO:== Results for 40 batches of

INFO:Starting epoch 5
INFO: == Evaluating on development data after batch 100
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.791, Recall: 0.791, F1: 0.791
INFO:-- Saving model
INFO: == Evaluating on development data after batch 200
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.829, Recall: 0.829, F1: 0.829
INFO:-- Saving model
INFO: == Evaluating on development data after batch 300
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.821, Recall: 0.821, F1: 0.821
INFO:-- Saving model
INFO: == Evaluating on development data after batch 400
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.834, Recall: 0.834, F1: 0.834
INFO:-- Saving model
INFO: == Evaluating on development data after batch 500
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.844, Recall: 0.844, F1: 0.844
INFO:-- Saving model
INFO: == Evaluating on devel

INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.909, Recall: 0.909, F1: 0.909
INFO:-- Saving model
INFO: == Evaluating on development data after batch 300
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.907, Recall: 0.907, F1: 0.907
INFO:-- Saving model
INFO: == Evaluating on development data after batch 400
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.905, Recall: 0.905, F1: 0.905
INFO:-- Saving model
INFO: == Evaluating on development data after batch 500
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.925, Recall: 0.925, F1: 0.925
INFO:-- Saving model
INFO: == Evaluating on development data after batch 600
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.922, Recall: 0.922, F1: 0.922
INFO:-- Saving model
INFO: == Evaluating on development data after batch 700
INFO:Ran 40 batches
INFO:== Results for 40 batches

## Evaluation

In [8]:
def print_example(pred, label, feats, length, email_id):
    ids_ = feats[:length][:, 0]
    print("==== {} ====".format(email_id))
    print("Predicted label: {}. True label: {}".format(LBLS[pred], LBLS[label]))
    print( " ".join((id2tok[i] for i in ids_)) )
    print("------------------------------------")

    

def print_examples(preds, outputs, inputs, lens, email_ids):
    for p, o, i, l, d in zip(preds, outputs, inputs, lens, email_ids):
        print_example(p, o, i, l, d)

In [12]:
preds, inputs, lens, outputs, email_ids, attns = model.predict()
correct = preds == outputs
print("Predicted {} of {} examples correctly (accuracy: {:.3f})".format(np.sum(correct), correct.shape[0], np.mean(correct)))

Predicted 31 of 32 examples correctly (accuracy: 0.969)


In [13]:
print_examples(preds, outputs, inputs, lens, email_ids)

==== b'<19436386.1075854910697.JavaMail.evans@thyme>' ====
Predicted label: NORESPONSE. True label: NORESPONSE
<s> good afternoon marie , here is a list of UUUNKKK company counterparties that require isda master agreements . i checked the financial trading agreements listing in lotus notes to make sure an isda master agreement did n't already exist . please double check your version in the event a counterparty was deleted as a result of the historical problem that we previously discussed regarding missing counterparty isda master references . counterparty name original trade date UUUNKKK de UUUNKKK UUUNKKK UUUNKKK UUUNKKK ect europe accounting UUUNKKK ectric crude UUUNKKK ectric UUUNKKK UUUNKKK ectric spain UUUNKKK ena merchant assets UUUNKKK enron broadband investments corporation UUUNKKK enron broadband services marketing ltd. UUUNKKK enron corporation UUUNKKK enron holdings NNNUMMM UUUNKKK . UUUNKKK enron industrial markets llc 07/13/01 enron international inc. UUUNKKK enron metals 

In [ ]:
import matplotlib


In [ ]:
def make_attn_plot(word_ids, attn):
    